# POS tagging

In [4]:
import nltk

In [13]:
text = nltk.word_tokenize("The quick brown fox jumped over the lazy dog and easily killed it")
# nltk.download('averaged_perceptron_tagger')
# nltk.download('tagsets')
#use a pos tagger
print("Using a POS tagger ====>  ")
print(nltk.pos_tag(text))
nltk.help.upenn_tagset('VBD')

Using a POS tagger ====>  
[('The', 'DT'), ('quick', 'JJ'), ('brown', 'NN'), ('fox', 'NN'), ('jumped', 'VBD'), ('over', 'IN'), ('the', 'DT'), ('lazy', 'JJ'), ('dog', 'NN'), ('and', 'CC'), ('easily', 'RB'), ('killed', 'VBD'), ('it', 'PRP')]
VBD: verb, past tense
    dipped pleaded swiped regummed soaked tidied convened halted registered
    cushioned exacted snubbed strode aimed adopted belied figgered
    speculated wore appreciated contemplated ...


In [47]:
freq = nltk.defaultdict(lambda: nltk.defaultdict(int))
freq['iit'] = 10
print(freq['nit'])
print(freq['iit'])

defaultdict(<class 'int'>, {})
10


<span style="color:blue"> <font size="4"> Tag of a word depends on the word and its context within a sentence</font> </span>




In [65]:
from nltk.corpus import brown
# nltk.download('brown')
brown_tagged_sents = brown.tagged_sents(categories='news')
brown_sents = brown.sents(categories='news')

# a default tagger (lets give the most frequent tag which is NN)
tokens = nltk.word_tokenize("The quick brown fox jumped over the lazy dog and easily killed it")
default_tagger = nltk.DefaultTagger("NN")
print("Default tagger assigns NN to all ===> \n", default_tagger.tag(tokens))
print('Default tagger acuracy :: {:.3f}'.format(default_tagger.evaluate(brown_tagged_sents)))

#get best 100 elements of the corpus with their most repeated tag 

fd = nltk.FreqDist(brown.words(categories='news'))
cfd = nltk.ConditionalFreqDist(brown.tagged_words(categories='news'))
most_freq_words = list(fd.keys())[:100] #get most frequent words
likely_tags = dict((word, cfd[word].max()) for word in most_freq_words) # get tags of these 
# print(list(cfd['of'].max()))
baseline_tagger = nltk.UnigramTagger(model=likely_tags,backoff=default_tagger) #if NOne do backoff
print("Baseline tagger assigns NN to all ===> \n", baseline_tagger.tag(tokens)) 
print('Baseline tagger acuracy :: {:.3f}'.format(baseline_tagger.evaluate(brown_tagged_sents)))

Default tagger assigns NN to all ===> 
 [('The', 'NN'), ('quick', 'NN'), ('brown', 'NN'), ('fox', 'NN'), ('jumped', 'NN'), ('over', 'NN'), ('the', 'NN'), ('lazy', 'NN'), ('dog', 'NN'), ('and', 'NN'), ('easily', 'NN'), ('killed', 'NN'), ('it', 'NN')]
Default tagger acuracy :: 0.131
Baseline tagger assigns NN to all ===> 
 [('The', 'AT'), ('quick', 'NN'), ('brown', 'NN'), ('fox', 'NN'), ('jumped', 'NN'), ('over', 'NN'), ('the', 'AT'), ('lazy', 'NN'), ('dog', 'NN'), ('and', 'CC'), ('easily', 'NN'), ('killed', 'NN'), ('it', 'PPS')]
Baseline tagger acuracy :: 0.461


## N gram Taggers 
Uses previous n-1 tags and current word to predict the current POS<br>
Problems with ngram taggers::<br>
    1. size of the n-gram table with backoffs is sparse and huge <br>
    2. only previous n-tags are conbsidered where there will be a lot of information in words itself


In [74]:
brown_tagged_sents = brown.tagged_sents(categories='news')
size = int(len(brown_tagged_sents) * 0.9)
train_sents = brown_tagged_sents[:size]
test_sents = brown_tagged_sents[size:]
bigram_tagger = nltk.BigramTagger(train_sents) #training on train sentences
print(bigram_tagger.tag(brown_sents[4300]))
print(bigram_tagger.evaluate(test_sents))
#performing worse as 'up' is tagged as None and in training None is not present in context and all subsequent words are None
##use backoffs for better results
t0 = nltk.DefaultTagger('NN')
t1 = nltk.UnigramTagger(train_sents, backoff=t0)
t2 = nltk.BigramTagger(train_sents, backoff=t1)
t3 = nltk.TrigramTagger(train_sents,backoff=t2)
print(t3.evaluate(test_sents))

[('High', 'JJ'), ('up', None), ('on', None), ('the', None), ("President's", None), ('priority', None), ('list', None), ('was', None), ('the', None), ('thorny', None), ('question', None), ('of', None), ('Bizerte', None), ('.', None)]
0.10206319146815508
0.843317053722715


## Brill tagging 

In [75]:
#hehaa